In [2571]:
using LinearAlgebra

In [2572]:
# Example usage:
A = randn(4, 6)

4×6 Matrix{Float64}:
  0.920341  -0.176348  -1.97011    2.18437   -0.264475     0.289831
 -0.639675   1.21815    0.259225   0.988614   0.00575467   1.61652
  0.557074  -1.63503   -1.1294    -0.528585   0.723477     0.836219
 -0.520058  -0.21672   -1.51848    1.26877   -1.10022     -1.0667

In [2573]:
function householder(x)
    """Computes the Householder transformation for input vector x
    
    Returns 
    beta: float, the multiplier for future Householder reflection
    v: vector, the householder reflector vector  
    """
    sigma = dot(x[2:end],x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq
    else
        v[1] -= sq
    end

    beta = 2.0 / (v[1]^2 + sigma)

    return beta, v
end


function apply_householder_col!(B, H, cidx, m)
    """Apply Householder reflection from the left (B<- H * B)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[cidx:m, cidx])
    B[cidx:m, cidx:end] = B[cidx:m, cidx:end] - beta * v * (v' * B[cidx:m, cidx:end])
    H[cidx:end, cidx] = v / norm(v)

    return beta
end

function apply_householder_row!(B, H, ridx, n)
    """Apply Householder reflection from the right (B<- B * H)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[ridx, ridx+1:n])
    B[ridx:end, ridx+1:n] = B[ridx:end, ridx+1:n] - (B[ridx:end, ridx+1:n] * v) * (beta * v')
    H[ridx, ridx+1:end] = v / norm(v)

end

function update_col_sim!(U, H, cidx, m)
    """Update column unitary transform (U<- H * U)
    
    Returns (inplace) 
    U: input matrix  
    """
    v = H[cidx:end, cidx]
    if cidx > 1
        v = [zeros(cidx-1);v]
    end
    #U[cidx:end, cidx:end] = U[cidx:end, cidx:end] - 2 * v * (v' * U[cidx:end, cidx:end])
    #show(stdout, "text/plain", I - 2 * v * v')
    U[:,:] = U - 2 * v * (v' * U)
    
    return U
    
end

function update_row_sim!(V, H, ridx, n)
    """Update row unitary transform (V<- V * H)
    
    Returns (inplace) 
    H: input matrix  
    """
    v = H[ridx, ridx+1:end]
    v = [zeros(ridx);v]
    
    
    #V[ridx:end, ridx+1:end] = V[ridx:end, ridx+1:end] - (V[ridx:end, ridx+1:n] * v) * (2 * v')
    #show(stdout, "text/plain", I - 2 * v * v')

    V[:,:] = V - 2 * (V * v) * v'
    
    return V
    
end


function bidiagonalize(A, return_orth = false)
    """Performs matrix bidiagonalization (for future singular value decomposition computation)
    
    Returns (inplace) 
    B: bidiagonal matrix with same singular values as input matrix (B = Q_r * A * Q_l)
    """
    m, n = size(A)
    B = copy(A)
    H = copy(A)  # v vectors in householder
    
    if return_orth
        U = zeros(m, m) + I
        V = zeros(n, n) + I
    end
    
    for k = 1:min(m,n)
        
        # column
        
        apply_householder_col!(B, H, k, m)
    
        
        if return_orth
            update_col_sim!(U, H, k, m)
        end
        
        
        # row
        if k < n
            apply_householder_row!(B, H, k, n) 
            
            if return_orth 
                update_row_sim!(V, H, k, n)
                #print("\ni = ",k,"\n")
                #show(stdout, "text/plain", B)
                #print("\n\n")
                #show(stdout, "text/plain", U * A * V)
            end
        end
        
                
    end
    
    if return_orth
        V[:, end] = -V[:, end] 
        U[end, :] = -U[end, :]
        return U', B, V
    else
        return B, H
    end
end



bidiagonalize (generic function with 2 methods)

In [2574]:
function givens_rotation(v)
    a, b = v[1], v[2]
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            tau = -a/b
            s = 1.0/sqrt(1.0+tau*tau)
            c = s*tau
        else
            tau = -b/a
            c = 1.0/sqrt(1.0+tau*tau)
            s = c*tau
        end
    end
    return [c -s;s c]
end


function svd_golub_reinsh!(B, U, V, maxiter, eps=1e-12)
    """Computes the Singular value decomposition of a bidiagonal matrix. Based on 
    https://www.cs.utexas.edu/~inderjit/public_papers/HLA_SVD.pdf
    
    Returns  
    U: Left singular vector matrix
    S: Singular values matrix
    Vt: Right singular vector matrix
    """
    
    function golub_kahan_step!(B, U, V, l, k)
        # Considering B22 -> (l:k x l:k)
        
        # Step 1: compute appropriate shift
        
        if k > 2
            a, b, d = B[k-1, k-1]^2 + B[k-2, k-1]^2, B[k-1, k-1] * B[k-1, k], B[k, k]^2 + B[k-1, k]^2
            t, r = - a - d, a * d - b^2
            s_1, s_2 = 0.5 * (-t + sqrt(t^2 -4 * r)), 0.5 * (-t - sqrt(t^2 -4 * r))
            if abs(B[k, k]^2 + B[k-1, k]^2 - s_1) < abs(B[k, k]^2 + B[k-1, k]^2 - s_2)
                s = s_1 
            else
                s = s_2 
            end
        else
           s = B[2, 2]^2 + B[1, 2]^2 
        end
        
        # Step 2: Computing right Givens Rotations
        alpha, beta = B[l, l]^2-s, B[l, l] * B[l, l + 1]
        v = [alpha;beta]
        for p = l:k-1
            # Right rotation
            G = givens_rotation(v)
            B[:, p:p+1] = B[:, p:p+1] * G'
            V[:, p:p+1] = V[:, p:p+1] * G'
            
            
            # Left rotation
            alpha, beta = B[p, p], B[p + 1, p] 
            v = [alpha; beta]
            G = givens_rotation(v)
            B[p:p+1, :] = G * B[p:p+1, :]
            U[:, p:p+1] = U[:, p:p+1] * G'
            
            # Updating alpha, beta and v
            if p < k -1 
                alpha, beta = B[p, p+1], B[p, p+2]
                v = [alpha;beta]
            end
        end
    end
    
    m, n = size(B)
    
    if (n > m)
        # To be done "fat" matrices
    end
        
    m, n = size(B)
    
    #U = zeros(m, m) + I
    #V = zeros(n, n) + I
    maxiter_ = maxiter * log2(n * m) 
    
    for iters = 1:min(maxiter, maxiter_)
        
        # Step 1 of main loop
        # Find l and k such that matrix can be blocked as B11, B22 and B33 (diag) 
        
        k = n
        for j = n:-1:2  
            #print(abs(B[j-1, j]), " ", abs(B[j-1, j]) < eps, "\n")
            k = abs(B[j-1, j]) < eps * (abs(B[j, j]) + abs(B[j-1, j-1]))  ? j-1 : k
        end
        #print("After scan k = ", k, "\n")
        
        
        #show(stdout, "text/plain", B)
        #print("\n---------------\n")
        
        if k == 1
           # Diagonal matrix, convergence obtained
           return U, S, V
        end
        
        if k == n 
            # B22 should be the full matrix 
            golub_kahan_step!(B, U, V, 1, n)
    
        else 
            # B33 (diagonal) found, have to check if where B22 starts (l) and ends (k)
            l = k
            for j = k:-1:2
                l = B[j-1, j] == 0 ? j-1 : l 
            end
            
            l = (l == k) ? 1 : l
            
            golub_kahan_step!(B, U, V, l, k)
        end
    end
end


function svd_num(A, maxiter = 80)
   """Performs singular value decompositon of a rectangular m x n matrix A.
    
    Returns 
        U: Left singular vector matrix
        S: Singular values matrix
        Vt: Right singular vector matrix
    """ 
    # Flag for transpose (case n > m)
    m, n = size(A)
    transpose_flag = false
    
    if n > m
        transpose_flag = true
    end
    
    # First main step: Find bidiagonalized matrix B
    if transpose_flag
        P_l, B, P_r = bidiagonalize(A', true);
    else
        P_l, B, P_r = bidiagonalize(A, true);
    end
    
    # Second main step: perform svd_golub_reinsh algorithm iterations
    svd_golub_reinsh!(B, P_l, P_r, maxiter)
    
    # Check if transpose
    if transpose_flag
        return P_r, B', P_l
    else
        return P_l, B, P_r
    end
    
    
    
end

svd_num (generic function with 2 methods)

In [2575]:
### TO DO: 
###1 - Create main SVD method which creates H matriz correctly for "fat matrices" (transpose before and fix after)

In [2576]:
A

4×6 Matrix{Float64}:
  0.920341  -0.176348  -1.97011    2.18437   -0.264475     0.289831
 -0.639675   1.21815    0.259225   0.988614   0.00575467   1.61652
  0.557074  -1.63503   -1.1294    -0.528585   0.723477     0.836219
 -0.520058  -0.21672   -1.51848    1.26877   -1.10022     -1.0667

In [2577]:
P_l, B, P_r = bidiagonalize(A, true);

In [2578]:
show(stdout, "text/plain", opnorm(P_l * B * P_r' - A, 1))

3.4416913763379853e-15

In [2579]:
P_l, B, P_r = bidiagonalize(A', true);

In [2580]:
P_l * B * P_r'

6×4 Matrix{Float64}:
  0.920341  -0.639675     0.557074  -0.520058
 -0.176348   1.21815     -1.63503   -0.21672
 -1.97011    0.259225    -1.1294    -1.51848
  2.18437    0.988614    -0.528585   1.26877
 -0.264475   0.00575467   0.723477  -1.10022
  0.289831   1.61652      0.836219  -1.0667

In [2581]:
U, S, V = svd_num(A);

In [2583]:
U

4×4 Matrix{Float64}:
 -0.806583   -0.521753  -0.275364  -0.0370398
 -0.0414493   0.424222  -0.590179  -0.68557
 -0.141391    0.438798  -0.514523   0.723002
 -0.572463    0.596041   0.557791  -0.0767448

In [2584]:
S

4×6 adjoint(::Matrix{Float64}) with eltype Float64:
  3.72211       0.0          -3.08907e-17  …   7.18758e-17   5.69304e-17
 -1.20367e-20   0.970269      2.90009e-18     -7.29338e-17   2.90128e-17
  5.02194e-19   2.05204e-17  -2.31185         -4.15208e-17  -2.17334e-16
 -7.57621e-16  -9.54869e-15  -1.11281e-13     -2.29133e-16  -3.26847e-17

In [2585]:
V

6×6 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.133491   -0.842126   0.195782    0.310612  -0.366745  -0.0607626
  0.120091   -0.245133  -0.0216314  -0.731161  -0.209182   0.588784
  0.700483   -0.270829  -0.0534699  -0.295079   0.18118   -0.559659
 -0.659422   -0.202022   0.088795   -0.454007   0.426225  -0.358714
  0.198981   -0.203951   0.396441    0.224898   0.727295   0.426741
  0.0514858   0.273818   0.89067    -0.15859   -0.279802  -0.160124

In [2586]:
show(stdout, "text/plain", opnorm(U * S * V' - A))

6.02393716246072e-15

In [2469]:
A = [1.0 3 4;4 -1 8;7 8 -1;8 0 1]

4×3 Matrix{Float64}:
 1.0   3.0   4.0
 4.0  -1.0   8.0
 7.0   8.0  -1.0
 8.0   0.0   1.0

In [2470]:
A

4×3 Matrix{Float64}:
 1.0   3.0   4.0
 4.0  -1.0   8.0
 7.0   8.0  -1.0
 8.0   0.0   1.0

In [2471]:
A

4×3 Matrix{Float64}:
 1.0   3.0   4.0
 4.0  -1.0   8.0
 7.0   8.0  -1.0
 8.0   0.0   1.0

In [2472]:
G = givens_rotation(A[[1,2], 1])

2×2 Matrix{Float64}:
 -0.242536  -0.970143
  0.970143  -0.242536

In [2473]:
A[[1,2], :] = G * A[[1,2], :];
A

4×3 Matrix{Float64}:
 -4.12311  0.242536  -8.73128
  0.0      3.15296    1.94029
  7.0      8.0       -1.0
  8.0      0.0        1.0

In [2474]:
G = givens_rotation(A[[1,3], 1])

2×2 Matrix{Float64}:
 0.507519  -0.86164
 0.86164    0.507519

In [2475]:
A[[1,3], :] = G * A[[1,3], :];
A


4×3 Matrix{Float64}:
 -8.12404      -6.77003  -3.56965
  0.0           3.15296   1.94029
 -3.33067e-16   4.26913  -8.03075
  8.0           0.0       1.0

In [2476]:
G = givens_rotation(A[[2,3], 2])

2×2 Matrix{Float64}:
 -0.594089  -0.8044
  0.8044    -0.594089

In [2477]:
A[[2,3], :] = G * A[[2,3], :];
A


4×3 Matrix{Float64}:
 -8.12404      -6.77003      -3.56965
  2.67919e-16  -5.30723       5.30723
  1.97871e-16   2.10348e-16   6.33174
  8.0           0.0           1.0

In [2478]:
qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor: 4×4 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}
R factor:
3×3 Matrix{Float64}:
 11.4018  4.82382   3.24511
  0.0     7.12255  -2.75938
  0.0     0.0      -7.99093

In [2479]:
B, H = bidiagonalize(A);

In [2480]:
show(stdout, "text/plain", B)

4×3 Matrix{Float64}:
 11.4018       -5.81378   0.0
  4.93038e-32  -6.24399   0.351945
  2.46519e-32   0.0      -9.1153
  1.77636e-15   0.0       0.0

In [2481]:
show(stdout, "text/plain", H)

4×3 Matrix{Float64}:
 -0.925345      0.956484   0.291786
  1.26969e-17   0.784476   1.0
  9.37729e-18  -0.360763   0.982398
  0.379127      0.504427  -0.186802

In [2482]:
_, S1, _ = svd(B);

In [2483]:
S1

3-element Vector{Float64}:
 13.175972258951573
  9.124625172206951
  5.39768195599424

In [2484]:
_, S2, _ = svd(A);

In [2485]:
S2

3-element Vector{Float64}:
 13.175972258951575
  9.12462517220695
  5.397681955994235

In [2486]:
P_l, B, P_r = bidiagonalize(A, true);

In [2487]:
# Checking correctness

In [2488]:
show(stdout, "text/plain", opnorm(B - P_l * A * P_r', 1)/opnorm(A, 1))

1.6720045410306188

In [2489]:
show(stdout, "text/plain", opnorm(P_r' * P_r -  I, 1))

2.220446049250313e-16

In [2490]:
show(stdout, "text/plain", opnorm(P_l' * P_l - I, 1))

1.1166580117841786e-15

In [2491]:
U, S, V = svd(B)

SVD{Float64, Float64, Matrix{Float64}, Vector{Float64}}
U factor:
4×3 Matrix{Float64}:
 -0.965355     -0.0238559    -0.259848
 -0.260777      0.0529289     0.963947
  0.00924234   -0.998313      0.0573162
 -1.12622e-16  -5.80319e-18  -1.80637e-16
singular values:
3-element Vector{Float64}:
 13.175972258951573
  9.124625172206951
  5.39768195599424
Vt factor:
3×3 Matrix{Float64}:
 -0.835364    0.549534   -0.0133596
 -0.0298093  -0.0210195   0.999335
 -0.548888   -0.835207   -0.0339402

In [2492]:
P_l' * U

4×3 Matrix{Float64}:
 0.68784    0.0169979   0.185148
 0.60148   -0.564035   -0.0457499
 0.315155   0.512966   -0.790456
 0.256466   0.646871    0.582071

In [2493]:
P_r' * V  

3×3 Matrix{Float64}:
 -0.835364  -0.0298093  -0.548888
 -0.448504  -0.540365    0.711934
 -0.317822   0.840903    0.438032

In [2494]:
svd(A)

SVD{Float64, Float64, Matrix{Float64}, Vector{Float64}}
U factor:
4×3 Matrix{Float64}:
 -0.831622  -0.0984949   0.356497
 -0.052638  -0.803398    0.269312
  0.15273   -0.583517   -0.513833
  0.531326  -0.0660222   0.732364
singular values:
3-element Vector{Float64}:
 13.175972258951575
  9.12462517220695
  5.397681955994235
Vt factor:
3×3 Matrix{Float64}:
 0.835364    0.448504   0.317822
 0.0298093   0.540365  -0.840903
 0.548888   -0.711934  -0.438032

In [2495]:
U' * P_l * A * P_r' * V   

3×3 Matrix{Float64}:
  5.52446    6.85996   -2.33767
 -5.76578   -0.998583  -0.347205
 -0.487225  -4.02701   -4.78307

In [2496]:
show(stdout, "text/plain", B)

4×3 Matrix{Float64}:
 11.4018       -5.81378   0.0
  4.93038e-32  -6.24399   0.351945
  2.46519e-32   0.0      -9.1153
  1.77636e-15   0.0       0.0